In [1]:
#Import Native Libraries
#import pandas as pd
#import numpy as np 
#import copy
from sklearn.metrics import classification_report
import pickle
#import matplotlib.pyplot as plt

In [2]:
#Import Needed Created Functions

#Need 2 Functions to extract all the data from csv files
from ExtractDataFunctions import ExtractAccelerometerData
from ExtractDataFunctions import ExtractAcousticData

#Need 3 Functions to Organize all the all the data
from OrganizationFunctions import Inputs2CondensedForm
from OrganizationFunctions import System2CondensedForm
from OrganizationFunctions import AllData2WorkingForm

#Need 1 Function to Organize Files into TestDataFrame
from FeatureFunctions import getTESTDataFrame

#Need 2 Functions for Graphing
from FeatureFunctions import getGraphs
from FeatureFunctions import getQuickPlot

#Need # Functions to perform Machine Learning
from MachineLearningFunctions import getTESTMatrix
from MachineLearningFunctions import GenerateTrainingFile
from MachineLearningFunctions import GetSplitTrainingData
from MachineLearningFunctions import GetTrainingData
from MachineLearningFunctions import TrainModel
from MachineLearningFunctions import PredictModel
from MachineLearningFunctions import PredictProbModel
from MachineLearningFunctions import GetReducedFeaturesFromDataFrame

In [3]:
#Receive GUI Inputs

#General
Name_ID = "1"
Application = "2"
ModelNumber = "3"
SavingAlias = "4"
#AccelerometerDataFilename = 'AccelerometerActualData.csv' #filename #Origninal / Additionally File
AccelerometerDataFilename = 'AccelerometerActualDataEdited.csv' #filename #main file
#AcousticDataFilename = 'DataOutputMic.csv' #Original File #Wave File Name
AcousticDataFilename = 'DataOutputMic2Col.csv' #Wave File Name
MLDataFilename = "MLSynthesizedData.csv"

#Motor Characteristics #Need to get with Brendan
Horsepower = "6"
RatedVoltage = "7"
ACorDC = "DC"
NumberOfPolePairs = "9"
NumberofShafts = "10"

#Bearing Information
ShaftSpeed = 300 #Also Used for Motor Characteristics
NumberOfRollingElements = 3
DiameterOfRollingElements = 3
PitchDiameter = .2
ContactAngle = .2

#Processing Information 
AccelerometerSamplingFrequency = 14000 #must be an non-zero int or float
AcousticSamplingFrequency = 20000 #must be an non-zero int or float

In [4]:
#Microcontroller Information
#Receive Required Information
A2DResolution = 16
VoltageMax = 5
VoltageMin = 0

In [5]:
#System/Sensor Known Constants
AccelerationMax = 50 
AccelerationMin = -50

In [6]:
#Convert User Inputs into a condensed form
OnlyUserInput = Inputs2CondensedForm(Name_ID, Application, ModelNumber, SavingAlias,\
                                     AccelerometerDataFilename, AcousticDataFilename, \
                                     MLDataFilename, Horsepower, RatedVoltage, ACorDC, \
                                     NumberOfPolePairs, NumberofShafts, \
                                     ShaftSpeed, NumberOfRollingElements, \
                                     DiameterOfRollingElements,PitchDiameter, ContactAngle, \
                                     AccelerometerSamplingFrequency, AcousticSamplingFrequency)

SystemInputs = System2CondensedForm(A2DResolution,VoltageMax,VoltageMin,AccelerationMax,AccelerationMin)

In [7]:
#Acquire Accelerometer Actual Data
time, amp, Voltage, Acceleration = ExtractAccelerometerData(OnlyUserInput,SystemInputs)

#Acquire Acoustic Actual Data
Channel1Time,Channel1Value,Channe21Time,Channe2Value = ExtractAcousticData(OnlyUserInput,SystemInputs)

In [8]:
#Put All Data into Working Form
trial = 2 #Select the instance of the data
AllData = AllData2WorkingForm(OnlyUserInput,SystemInputs,time[trial], \
                              amp[trial], Voltage[trial], Acceleration[trial],\
                             Channel1Time,Channel1Value,Channe21Time,Channe2Value)

In [9]:
#Prepare for Machine Learning
TestDF = getTESTDataFrame(AllData)
TestMatrix = getTESTMatrix(TestDF)
ShortDF = GetReducedFeaturesFromDataFrame(TestDF)
ShortTestMatrix = getTESTMatrix(ShortDF)

In [10]:
"""
GENERATE TRAINING DATA FILE
"""
#GenerateTrainingFile(AllData,"MLSynthesizedData.csv")
print('finished')

finished


In [11]:
"""
GET TRAINING DATA
"""

#Split training datasets for validation
#Used in scoring
X_train, X_test, Y_train, Y_test = GetSplitTrainingData(AllData)

#For trainging over entire file
#Used in final classifier
#Not used in scoring
Xall_train, Yall_train, dataset = GetTrainingData(AllData)

#Entire Set with Reduced Features
ShortDataset = GetReducedFeaturesFromDataFrame(dataset)
ShortXallTrain = ShortDataset.values[:,0:(ShortDataset.shape[1]-1)]
ShortYallTrain = Yall_train


print('finished')

finished


In [12]:
"""
MODEL VALIDATION

Demonstrate Selected Model's results using
classification report

Test on the split model
"""

#Train
Classifier = TrainModel(X_train, Y_train)


#Predict
Y_pred,Y_pred_string = PredictModel(Classifier,X_test)

print(Y_pred)
print(Y_pred_string)

#Output Results
#
print('\n')
print(classification_report(Y_test,Y_pred))

print('\nfinished')

[0. 1. 1. 0. 0. 2. 0. 1. 1. 0. 0. 1. 0. 1. 1.]
['Suspect', 'Normal', 'Normal', 'Suspect', 'Suspect', 'Failure', 'Suspect', 'Normal', 'Normal', 'Suspect', 'Suspect', 'Normal', 'Suspect', 'Normal', 'Normal']


             precision    recall  f1-score   support

        0.0       0.43      0.60      0.50         5
        1.0       0.14      0.50      0.22         2
        2.0       0.00      0.00      0.00         8

avg / total       0.16      0.27      0.20        15


finished


In [13]:
"""
SAVING FINAL TRAINED MODELS

Train on Final Model using all of the training data
"""
#Save Full Model
FinalClassifier = TrainModel(Xall_train, Yall_train)
SaveFilename = 'FullModel.sav' #saving name
pickle.dump(FinalClassifier, open(SaveFilename, 'wb')) # Save Model Using Pickle

#Save Short Model
ShortClassifier = TrainModel(ShortXallTrain,ShortYallTrain)
ShortSaveFilename = 'ShortModel.sav'
pickle.dump(ShortClassifier, open(ShortSaveFilename, 'wb'))

print('finished')

finished


In [14]:
"""
PREDICTION RESULTS on TestMatrix/ShortTestMatrix
"""

#Currently Trained Model
prediction,prediction_string = PredictModel(FinalClassifier,TestMatrix)
prediction_proba = PredictProbModel(FinalClassifier,TestMatrix)
#Output Results
print('Full Model:\n')
print(prediction_string)
print(prediction_proba)


#Current Short Classifier
prediction,prediction_string = PredictModel(ShortClassifier,ShortTestMatrix)
prediction_proba = PredictProbModel(ShortClassifier,ShortTestMatrix)
#Output Results
print('\n')
print('Short Model:\n')
print(prediction_string)
print(prediction_proba)


#Current Saved Loaded Model
SaveFilename = 'FullModel.sav' #saving name
LoadedClassifier = pickle.load(open(SaveFilename, 'rb')) #Load the saved model using Pickle
#Predict
prediction,prediction_string = PredictModel(LoadedClassifier,TestMatrix)
prediction_proba = PredictProbModel(LoadedClassifier,TestMatrix)
#Output Results
print('\n')
print('Loaded Model:\n')
print(prediction_string)
print(prediction_proba)


#Current Saved ShortModel
ShortSaveFilename = 'ShortModel.sav' #saving name
LoadedClassifier = pickle.load(open(ShortSaveFilename, 'rb')) #Load the saved model using Pickle
prediction,prediction_string = PredictModel(LoadedClassifier,ShortTestMatrix)
prediction_proba = PredictProbModel(LoadedClassifier,ShortTestMatrix)
#Output Results
print('\n')
print('Loaded Short Model:\n')
print(prediction_string)
print(prediction_proba)


print('\nfinished')

"""END OF MODEL CHARACTERIZING"""

Full Model:

['Failure']
[[18.00449408 13.01237149 68.98313443]]


Short Model:

['Failure']
[[17.43682425 16.50400412 66.05917163]]


Loaded Model:

['Failure']
[[18.00449408 13.01237149 68.98313443]]


Loaded Short Model:

['Failure']
[[17.43682425 16.50400412 66.05917163]]

finished


'END OF MODEL CHARACTERIZING'